In [1]:
import time
import os
import hashlib
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes

In [2]:
def generate_aes_key(key_size):
    return get_random_bytes(key_size // 8)

In [3]:
def aes_encrypt(data, key, mode):
    if mode == AES.MODE_ECB:
        cipher = AES.new(key, mode)
        ciphertext = cipher.encrypt(data)
        return ciphertext, b''
    else:
        iv = get_random_bytes(AES.block_size)
        cipher = AES.new(key, mode, iv=iv)
        ciphertext = cipher.encrypt(data)
        return ciphertext, iv

In [4]:
def aes_decrypt(ciphertext, key, mode, iv):
    if mode == AES.MODE_ECB:
        cipher = AES.new(key, mode)
        return cipher.decrypt(ciphertext)
    else:
        cipher = AES.new(key, mode, iv=iv)
        return cipher.decrypt(ciphertext)

In [5]:
def generate_rsa_keypair(bits):
    key = RSA.generate(bits)
    return key, key.publickey()

In [6]:
def rsa_encrypt(data, public_key):
    cipher = PKCS1_OAEP.new(public_key)
    return cipher.encrypt(data)

In [7]:
def rsa_decrypt(ciphertext, private_key):
    cipher = PKCS1_OAEP.new(private_key)
    return cipher.decrypt(ciphertext)

In [8]:
def rsa_sign(data, private_key):
    h = SHA256.new(data)
    signature = pkcs1_15.new(private_key).sign(h)
    return signature

In [9]:
def rsa_verify(data, signature, public_key):
    h = SHA256.new(data)
    try:
        pkcs1_15.new(public_key).verify(h, signature)
        return True
    except (ValueError, TypeError):
        return False

In [10]:
def sha256_hash(data):
    return hashlib.sha256(data).hexdigest()

In [11]:
def save_to_file(filename, data):
    with open(filename, 'wb') as f:
        f.write(data)

In [12]:
def read_from_file(filename):
    with open(filename, 'rb') as f:
        return f.read()

In [ ]:
def main():
    os.makedirs('keys', exist_ok=True)
    os.makedirs('encrypted', exist_ok=True)
    os.makedirs('signatures', exist_ok=True)

    while True:
        print("Select an option:")
        print("1. AES Encryption/Decryption")
        print("2. RSA Encryption/Decryption")
        print("3. RSA Signature")
        print("4. SHA-256 Hashing")
        print("5. Exit")
        choice = int(input("Enter choice: "))

        if choice == 1:
            key_size = int(input("Enter AES key size (128 or 256): "))
            mode = int(input("Enter mode (1 for ECB, 2 for CFB): "))
            mode = AES.MODE_ECB if mode == 1 else AES.MODE_CFB
            key = generate_aes_key(key_size)
            save_to_file(f'keys/aes_key_{key_size}.key', key)
            data = input("Enter data to encrypt: ").encode()
            padded_data = data + b' ' * (16 - len(data) % 16)  # Padding for AES
            start_time = time.time()
            ciphertext, iv = aes_encrypt(padded_data, key, mode)
            end_time = time.time()
            save_to_file('encrypted/aes_encrypted.bin', ciphertext)
            print(f"AES Encrypted data saved to 'encrypted/aes_encrypted.bin'")
            print(f"Encryption took {end_time - start_time:.4f} seconds")
            decrypted_data = aes_decrypt(ciphertext, key, mode, iv)
            print(f"Decrypted data: {decrypted_data.strip()}")

        elif choice == 2:
            bits = int(input("Enter RSA key size (e.g., 1024, 2048): "))
            private_key, public_key = generate_rsa_keypair(bits)
            save_to_file(f'keys/rsa_private_{bits}.pem', private_key.export_key())
            save_to_file(f'keys/rsa_public_{bits}.pem', public_key.export_key())
            data = input("Enter data to encrypt: ").encode()
            start_time = time.time()
            ciphertext = rsa_encrypt(data, public_key)
            end_time = time.time()
            save_to_file('encrypted/rsa_encrypted.bin', ciphertext)
            print(f"RSA Encrypted data saved to 'encrypted/rsa_encrypted.bin'")
            print(f"Encryption took {end_time - start_time:.4f} seconds")
            decrypted_data = rsa_decrypt(ciphertext, private_key)
            print(f"Decrypted data: {decrypted_data}")

        elif choice == 3:
            bits = int(input("Enter RSA key size for signing (e.g., 1024, 2048): "))
            private_key, public_key = generate_rsa_keypair(bits)
            save_to_file(f'keys/rsa_private_sign_{bits}.pem', private_key.export_key())
            save_to_file(f'keys/rsa_public_sign_{bits}.pem', public_key.export_key())
            filename = input("Enter the filename to sign: ")
            data = read_from_file(filename)
            start_time = time.time()
            signature = rsa_sign(data, private_key)
            end_time = time.time()
            save_to_file('signatures/rsa_signature.sig', signature)
            print(f"Signature saved to 'signatures/rsa_signature.sig'")
            print(f"Signing took {end_time - start_time:.4f} seconds")
            valid = rsa_verify(data, signature, public_key)
            print(f"Signature valid: {valid}")

        elif choice == 4:
            filename = input("Enter the filename to hash: ")
            data = read_from_file(filename)
            hash_value = sha256_hash(data)
            print(f"SHA-256 Hash: {hash_value}")

        elif choice == 5:
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


Select an option:
1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. RSA Signature
4. SHA-256 Hashing
5. Exit
Enter choice: 2
Enter RSA key size (e.g., 1024, 2048): 2048
Enter data to encrypt: During your career, you may need to write a document explaining your skills, abilities and qualifications. 
RSA Encrypted data saved to 'encrypted/rsa_encrypted.bin'
Encryption took 0.0031 seconds
Decrypted data: b'During your career, you may need to write a document explaining your skills, abilities and qualifications. '
Select an option:
1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. RSA Signature
4. SHA-256 Hashing
5. Exit
Enter choice: 3
Enter RSA key size for signing (e.g., 1024, 2048): 1024
Enter the filename to sign: input.txt
Signature saved to 'signatures/rsa_signature.sig'
Signing took 0.0020 seconds
Signature valid: True
Select an option:
1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. RSA Signature
4. SHA-256 Hashing
5. Exit
Enter choice: 4
Enter t